## E+ in zeppy (using TCP)
Here we try to run E+ thru zeppy

Lets start by using eppy to do this
(make sure you have eppy installed before you do this.)

In [1]:
# setting the path name so that the tutorial works from this directory
# if you install zeppy with `pip install zeppy` you will not need to do this
import sys
# pathnameto_zeppy = 'c:/eppy'
pathnameto_zeppy = '../../'
sys.path.append(pathnameto_zeppy)

In [2]:
import os
import eppy
from zeppy import ptcp
wfile = "/Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw"

In [3]:
fnames = [
    # "./eplus_files/Minimal.idf",
        "../../eplus_files/UnitHeaterGasElec.idf",
        "../../eplus_files/ZoneWSHP_wDOAS.idf",
        "../../eplus_files/ZoneWSHP_wDOAS_1.idf",
        ]
wfile = "/Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw"
idfs = [eppy.openidf(fname, epw=wfile) for fname in fnames]

Let us write a couple of helper functions

In [4]:
def idf_multirun(idf_kwargs):
    import eppy
    # for some reason idf.run() does not work
    # so I am using runIDFs to run one idf at a time
    idf = idf_kwargs['args']
    options = idf_kwargs['kwargs']
    eppy.runner.run_functions.runIDFs( [(idf, options)] ) 


def make_options(idf):
    idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
    idfversion.extend([0] * (3 - len(idfversion)))
    idfversionstr = '-'.join([str(item) for item in idfversion])
    fname = idf.idfname
    options = {
        'ep_version':idfversionstr,
        'output_prefix':os.path.basename(fname).split()[0],
        'output_suffix':'C',
        'output_directory':os.path.dirname(fname),
        }
    return options


Now let us run the E+ files

In [5]:
waitlist = [[{'args':idf, 'kwargs':make_options(idf)}] for idf in idfs]
func = ptcp.idf_multirun
result = ptcp.ipc_parallelpipe(func, waitlist, nworkers=None, verbose=True)
print(result)

started worker 0
started worker 1
started worker 2
starting sink
started ventilator
EnergyPlus Starting
EnergyPlus, Version 9.1.0-08d2e308bb, YMD=2022.12.01 07:44
EnergyPlus Starting
EnergyPlus, Version 9.1.0-08d2e308bb, YMD=2022.12.01 07:44
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
EnergyPlus Starting
EnergyPlus, Version 9.1.0-08d2e308bb, YMD=2022.12.01 07:44
Initializing Response Factors
Calculating CTFs for "EXT-WALLS", Construction # 1
Calculating CTFs for "FLOOR", Construction # 2
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variabl

Writing final SQL reports
EnergyPlus Run Time=00hr 00min  2.09sec
Updating Shadowing Calculations, Start Date=04/11/2013
Continuing Simulation at 04/11/2013 for ANNUAL


EnergyPlus Completed Successfully.


Updating Shadowing Calculations, Start Date=04/11/2013
Continuing Simulation at 04/11/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=05/01/2013
Continuing Simulation at 05/01/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=05/01/2013
Continuing Simulation at 05/01/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=05/21/2013
Continuing Simulation at 05/21/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=05/21/2013
Continuing Simulation at 05/21/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=06/10/2013
Continuing Simulation at 06/10/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=06/10/2013
Continuing Simulation at 06/10/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=06/30/2013
Continuing Simulation at 06/30/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=06/30/2013
Continuing Simulation at 06/30/2013 for ANNUAL
Updating Shadowing Calculations, Start Date=07/20/2013
Continuing Simulation at 07

EnergyPlus Completed Successfully.
EnergyPlus Completed Successfully.


running item: 1, in worker: 1

/Applications/EnergyPlus-9-1-0/energyplus --weather /Applications/EnergyPlus-9-1-0/WeatherData/USA_CO_Golden-NREL.724666_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/github/zeppy/eplus_files --output-prefix ZoneWSHP_wDOAS.idf --output-suffix C /Users/santoshphilip/Documents/coolshadow/github/zeppy/eplus_files/ZoneWSHP_wDOAS_b6bc3a.idf

sent result of item: 1, in worker: 1 to sink


### Ha! all three files run. 
The results are `None`, since we did not write code to exract and return any outputs from the E+ outputs

Let us try running on a single worker and see how long it takes:

In [6]:
result = ptcp.ipc_parallelpipe(func, waitlist, nworkers=1, verbose=True)
print(result)

started worker 0
starting sink
started ventilator
EnergyPlus Starting
EnergyPlus, Version 9.1.0-08d2e308bb, YMD=2022.12.01 07:44
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 01/21 for CHICAGO ANN HTG 99% CONDNS DB
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 07/21 for CHICAGO ANN CLG 1% CONDNS DB=>MWB
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 01/01/2013 for RUNPERIOD 1
Updating Shadowing Calculations, Start Date=01/21/2013
Continuing Simulation at 01/21/2013 for RUNPERIOD 1
Updating Shadowing Calculations, Start Date=02/10/2

EnergyPlus Completed Successfully.


Initializing Response Factors
Calculating CTFs for "EXT-WALLS", Construction # 1
Calculating CTFs for "FLOOR", Construction # 2
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Computing Interior Diffuse Solar Exchange through Interzone Windows
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Convection Coefficients
Gathering Information for Predefi

EnergyPlus Completed Successfully.


Initializing Response Factors
Calculating CTFs for "EXT-WALLS", Construction # 1
Calculating CTFs for "FLOOR", Construction # 2
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Computing Interior Diffuse Solar Exchange through Interzone Windows
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Distribution
Initializing Interior Convection Coefficients
Gathering Information for Predefi

EnergyPlus Completed Successfully.


OK. Took a bit longer running on just one worker

### Multi core machines

- some superficial testing has shown that on a multicore machine, each run is on a different core.
- you can force the number of cores running by setting a value for `nworkers`
- if you have more runs than cores. Say you have 4 cores and 50 runs, you can set `nworker=4`. The runs would be spread evenly on all the cores. About 12 or 13 runs on each core. Each core will execute the runs sequentially.
- If you have 4 cores and assign 10 workers, it will still run. We have not checked ot see how it does it.